In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 2 : experimentation: get started with Vertex AI Training for R

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_training_r.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 2 : experimentation: get started with Vertex AI Training for R. Please note that this notebook should be ran only in R notebook image (e.g., R4.1).

### Objective

In this tutorial, you learn how to use `Vertex AI Training` for training a R custom model.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Training`
- `Vertex AI Model` resource

The steps performed include:

- Locally train an R model in a notebook using %%R magic commands
- Create a deployment image with trained R model and serving functions.
- Test the deployment image locally.
- Create a `Vertex AI Model` resource for the deployment image with embedded R model.
- Deploy the deployment image with embedded R model to a `Vertex AI Endpoint` resource.
- Test the deployment image with embedded R model.
- Create a R-to-Python training package.
- Create a training image for training the model.
- Train a R model using `Vertex AI Trainingh` service with the R-to-Python training package.

### Dataset

The dataset used for this tutorial is the Iris dataset built into the R package. This dataset does not require any feature engineering. The trained model predicts the type of Iris flower species from a class of three species: setosa, virginica, or versicolor.

### Costs 


This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installations

Install the packages required for executing this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform[tensorboard] $USER_FLAG -q
! pip3 install --upgrade rpy2 $USER_FLAG -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}
if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

1. **Click Create service account**.

2. In the **Service account name** field, enter a name, and click **Create**.

3. In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex AI" into the filter box, and select **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

4. Click Create. A JSON file that contains your key downloads to your local environment.

5. Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex AI SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [ ]:
import traceback

import google.cloud.aiplatform as aip

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### Set hardware accelerators

You can set hardware accelerators for training and prediction.

Set the variables `TRAIN_GPU/TRAIN_NGPU` and `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Telsa K80 GPUs allocated to each VM, you would specify:

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)


Otherwise specify `(None, None)` to use a container image to run on a CPU.

Learn more about [hardware accelerator support for your region](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators).

*Note*: TF releases before 2.3 for GPU support will fail to load the custom model in this tutorial. It is a known issue and fixed in TF 2.3. This is caused by static graph ops that are generated in the serving function. If you encounter this issue on your own custom models, use a container image for TF 2.3 with GPU support.

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### Set machine type

Next, set the machine type to use for training and prediction.

- Set the variables `TRAIN_COMPUTE` and `DEPLOY_COMPUTE` to configure  the compute resources for the VMs you will use for for training and prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU.
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: The following is not supported for training:*

 - `standard`: 2 vCPUs
 - `highcpu`: 2, 4 and 8 vCPUs

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## Introduction to R training

### Training a R model

You can either train your model locally, or use the `Vertex AI Training` service. In the later case, you would install the R-to-Python interpreter `rpy2` into your training instance (e.g., setup.py) and execute the R training script using the R-to-Python interpreter `rpy2`.


### Deploying a R model

Deploying a R model on `Vertex AI Prediction` service requires to use a custom container that serves online predictions. In this tutorial, you deploy a container running plumber R package to serve predictions from trained model artifacts.

### Load the R notebook interpreter

Loading the module `rpy2.ipython` will add support for %R and %RR magic cells in your notebook.

In [ ]:
%load_ext rpy2.ipython

### Install and import some additional R packages

In [ ]:
%%R

install.packages(c("randomForest", "plumber"), repos = "http://cran.us.r-project.org")

library(ggplot2)
library(randomForest)
library(plumber)

#### Quick peek at your data

This tutorial uses a version of the iris dataset that is built into the R package.

Start by doing a quick peek at the data.

In [ ]:
%%R

head(iris)

In [ ]:
# Make folder for R
! rm -rf deploy custom
! mkdir deploy custom custom/trainer

### Locally train an R model

First, you locally train the R model.

In [ ]:
%%R

# train model
model = randomForest(Species ~ ., data = iris)
# save model
save(model, file = "deploy/model.RData")

### Serving script

You create a R script for serving predictions. This script does the following:

- Extracts prediction request values from the HTTP body of the input request.
- Construct a prediction request for the R model.
- Submit the prediction request to the R model.
- Returns the predicted results.

*Note*: Plumber makes use of comment “annotations” above functions to define the web service. When you feed the file into Plumber, you’ll get a runnable web service that other systems can interact with over a network.

In [ ]:
%%writefile deploy/serving.R
# serving.R

library("randomForest")

#* Health check
#* @get /ping
#* @serializer unboxedJSON
function() {
    list(status = "OK")
}

#* @apiTitle flower classifier
#* @param petal_length
#* @param petal_width
#* @param sepal_length
#* @param sepal_width
#* @post /classify
function (req)
{
    instances <- as.data.frame(jsonlite::fromJSON(req$postBody))
    results <- list()

    load("./model.RData")

    for(i in 1:nrow(instances)) {       # for-loop over columns
        petal_length <- instances[i, "instances.petal_length"]
        petal_width <- instances[i, "instances.petal_width"]
        sepal_length <- instances[i, "instances.sepal_length"]
        sepal_width <- instances[i, "instances.sepal_width"]
        test = c(sepal_length, sepal_width, petal_length, petal_width)
        test = sapply(test, as.numeric)
        test = data.frame(matrix(test, ncol = 4))
        colnames(test) = colnames(iris[, 1:4])
        results <- append(results, predict(model, test))
    }

    list(predictions = results)
}

#### Script for running the R server

Next, you create a file that runs the server.

In [ ]:
%%writefile deploy/startServer.R
library(plumber)
pr <- plumb("serving.R")
pr$run(host = "0.0.0.0", port = 7080)

### Make R container for prediction

Currently, Vertex AI does not have a prefined container for making predictions with a deployed R model. No problem, you can assemble your own custom container. For this tutorial, you construct a deployment container from a Docker image as follows:

    - Set the base image supplied by RStudio (rstudio/plumber).
    - Package the model artifacts and serving scripts into a Docker image.
    - Start the serving script on port 7080.

In [ ]:
%%writefile deploy/Dockerfile

FROM rstudio/plumber

# install random forest
RUN R -e 'install.packages(c("randomForest"), repos = "https://cran.rstudio.com/")'

# Copy model and script
RUN mkdir /app
COPY model.RData /app
COPY serving.R /app
COPY startServer.R /app
WORKDIR /app

# plumber & run server
EXPOSE 7080

ENTRYPOINT ["R", "-f", "/app/startServer.R"]

In [ ]:
DEPLOY_IMAGE = f"gcr.io/{PROJECT_ID}/r-predict-iris"
print(DEPLOY_IMAGE)

! docker build --tag=$DEPLOY_IMAGE ./deploy

! docker push $DEPLOY_IMAGE

### Locally test the Docker image

Next, you locally test the Docker image you created for serving predictions.

#### Launch the serving binary

First, you launch the serving binary, listening on port 7080, and then ping it as a health check.

In [ ]:
! docker stop local_iris 2>/dev/null
! docker run -t -d --rm -p 7080:7080 --name=local_iris $DEPLOY_IMAGE
! sleep 10
! curl http://localhost:7080/ping

#### Send predicton request

Next, you send a prediction request to the serving binary you locally launched. Afterwards, you will shutdown the launched serving binary.

In [ ]:
%%bash

cat > ./deploy/instances.json <<END
{
  "instances": [{
      "sepal_width": 1,
      "sepal_length": 2,
      "petal_width": 3,
      "petal_length": 1
    },
    {
      "sepal_width": 4,
      "sepal_length": 2,
      "petal_width": 1,
      "petal_length": 1
    }
  ]
}
END

curl -s -X POST \
  -H "Content-Type: application/json; charset=utf-8" \
  -d @./deploy/instances.json \
  http://localhost:7080/classify

docker stop local_iris 1>/dev/null

### Upload a R model to a `Vertex AI Model` resource

Next you upload the R model to a Vertex AI Model resource, with the following parameters:

- `display_name`: A human readable name for the model resource.
- `serving_container_image_uri`: The deployment image that contains the serving binary and R model.
- `serving_container_predict_route`: The URI endpoint for prediction requests.
- `serving_container_health_route`: The URI endpoint for health ping.
- `serving_container_ports`: A list of ports to listen on for prediction/health requests.

In [ ]:
DISPLAY_NAME = "iris_" + TIMESTAMP
health_route = "/ping"
predict_route = "/classify"
serving_container_ports = [7080]

model = aip.Model.upload(
    display_name=DISPLAY_NAME,
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports,
)

model.wait()

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method, with the following parameters:

- `deployed_model_display_name`: A human readable name for the deployed model.
- `traffic_split`: Percent of traffic at the endpoint that goes to this model, which is specified as a dictionary of one or more key/value pairs.
If only one model, then specify as { "0": 100 }, where "0" refers to this model being uploaded and 100 means 100% of the traffic.
If there are existing models on the endpoint, for which the traffic will be split, then use model_id to specify as { "0": percent, model_id: percent, ... }, where model_id is the model id of an existing model to the deployed endpoint. The percents must add up to 100.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `starting_replica_count`: The number of compute instances to initially provision.
- `max_replica_count`: The maximum number of compute instances to scale to. In this tutorial, only one instance is provisioned.

In [ ]:
DEPLOYED_NAME = "iris-" + TIMESTAMP

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1

if DEPLOY_GPU:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        accelerator_type=DEPLOY_GPU.name,
        accelerator_count=DEPLOY_NGPU,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )
else:
    endpoint = model.deploy(
        deployed_model_display_name=DEPLOYED_NAME,
        traffic_split=TRAFFIC_SPLIT,
        machine_type=DEPLOY_COMPUTE,
        min_replica_count=MIN_NODES,
        max_replica_count=MAX_NODES,
    )

### Make test prediction

Next, you test the deployed model by sending synthentic data.

In [ ]:
try:
    INSTANCES = [
        {"sepal_width": 1, "sepal_length": 2, "petal_width": 3, "petal_length": 1},
        {"sepal_width": 4, "sepal_length": 2, "petal_width": 1, "petal_length": 1},
    ]

    prediction = endpoint.predict(instances=INSTANCES)

    print(prediction)
except:
    traceback.print_exc()

## Undeploy the model

When you are done doing predictions, you undeploy the model from the `Endpoint` resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.undeploy_all()

#### Delete the endpoint

The method 'delete()' will delete the endpoint.

In [ ]:
endpoint.delete()

#### Delete the model

The method 'delete()' will delete the model.

In [ ]:
model.delete()

### Create the task script for the Python/R training package

Next, you create the `task.py` script for driving the training package. Some noteable steps include:


- Command-line arguments:
    - `model-dir`: The location to save the trained model. When using Vertex AI custom training, the location will be specified in the environment variable: `AIP_MODEL_DIR`.


- Training:
    - Uses R-to-Python interpreter to run the R training script.


- Model artifact saving:
    - Saves the model artifacts at the Cloud Storage location specified by `model-dir`.
    - *Note*: GCSFuse (`/gcs`) is used to do filesystem operations on Cloud Storage buckets.

In [ ]:
%%writefile custom/trainer/task.py

import os
import sys
import rpy2
import argparse
import logging

# import rpy2's package module
import rpy2.robjects.packages as rpackages

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

# import R's utility package
randomForest = rpackages.importr('randomForest')

# train the model
logging.info("Model training started ...")
rpy2.robjects.r('''
    # train model
    model = randomForest(Species ~ ., data = iris)
    # save model
    save(model, file = "model.RData")
'''
)
logging.info("Model training completed ...")

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

# Upload the saved model file to Cloud Storage
gcs_model_path = os.path.join(args.model_dir, 'model.RData')
logging.info("Saving model artifacts to {}". format(gcs_model_path))
with open("model.RData", "rb") as f:
    data = f.read()
with open(gcs_model_path, "wb") as f:
    f.write(data)

### Make R container for training

Currently, Vertex AI does not have a prefined container for training an R model. No problem, you can assemble your own custom container. For this tutorial, you construct a deployment container from a Docker image as follows:

    - Set the base image to a TensorFlow Deep Learning image
    - Install R-to-Python package and other R tools
    - Install cloud storage package
    - Package the model artifacts and serving scripts into a Docker image.
    - Set the entry point in the container to run the training package

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-3

RUN apt-get update && apt-get install -y --no-install-recommends build-essential r-base r-cran-randomforest python3.6 python3-pip python3-setuptools python3-dev
RUN pip install google-cloud-storage
RUN pip install rpy2

# install random forest
RUN R -e 'install.packages(c("randomForest"), repos = "https://cran.rstudio.com/")'

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

In [ ]:
TRAIN_IMAGE = f"gcr.io/{PROJECT_ID}/r-train-iris"
print(TRAIN_IMAGE)

! docker build --tag=$TRAIN_IMAGE ./custom

! docker push $TRAIN_IMAGE

### Create and run custom training job


To train a custom model, you perform two steps: 1) create a custom training job, and 2) run the job.

#### Create custom training job

A custom training job is created with the `CustomTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the custom training job.
- `container_uri`: The training container image.

- `command`: The command (e.g., interpreter) and script to invokee within the container.

*Note:* The interpreter and script to invoke is overridable within the container (i.e., ENTRYPOINT).

In [ ]:
job = aip.CustomContainerTrainingJob(
    display_name="iris_" + TIMESTAMP,
    container_uri=TRAIN_IMAGE,
    command=["python3", "trainer/task.py"],
)

print(job)

#### Run the custom container training job

Next, you run the custom job to start the training job by invoking the method `run()`. The parameters are the same as when running a CustomTrainingJob.

In [ ]:
CMDARGS = ["--model-dir=" + BUCKET_URI]

job.run(args=CMDARGS, replica_count=1, machine_type=TRAIN_COMPUTE, sync=True)

### Delete a custom training job

After a training job is completed, you can delete the training job with the method `delete()`.  Prior to completion, a training job can be canceled with the method `cancel()`.

In [ ]:
job.delete()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Model (Already deleted in previous cells)
- Endpoint (Already deleted in previous cells)
- Custom Job (Already deleted in previous cells)
- Cloud Storage Bucket

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI